In [ ]:
import numpy as np
from my_functions import *
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

def load_stuff():
    w_central = central_wavelength()

    pm_flx = np.array([]).reshape(60, 0)
    pm_err = np.array([]).reshape(60, 0)
    tile_id = np.array([])
    parallax_sn = np.array([])
    pmra_sn = np.array([])
    pmdec_sn = np.array([])
    starprob = np.array([])
    spCl = np.array([])
    zsp = np.array([])
    N_minijpas = 0
    for name in ['minijpas', 'jnep']:
        cat = pd.read_csv(f'csv/{name}.Flambda_aper3_photoz_gaia_3.csv', sep=',', header=1,
            converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
            5: sum_flags})

        cat = cat[np.array([len(x) for x in cat['FLUX_APER_3_0']]) != 0] # Drop bad rows due to bad query
        cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
        cat = cat.reset_index()

        tile_id_i = cat['TILE_ID'].to_numpy()

        parallax_i = cat['parallax'].to_numpy() / cat['parallax_error'].to_numpy()
        pmra_i = cat['pmra'].to_numpy() / cat['pmra_error'].to_numpy()
        pmdec_i = cat['pmdec'].to_numpy() / cat['pmdec_error'].to_numpy()

        pm_flx_i = np.stack(cat['FLUX_APER_3_0'].to_numpy()).T * 1e-19
        pm_err_i = np.stack(cat['FLUX_RELERR_APER_3_0'].to_numpy()).T * pm_flx_i

        if name == 'minijpas':
            N_minijpas = pm_flx_i.shape[1]

        starprob_i = cat['morph_prob_star']

        pm_err_i = (pm_err_i ** 2 + Zero_point_error(cat['TILE_ID'], name) ** 2) ** 0.5

        spCl_i = cat['spCl']
        zsp_i = cat['zsp']

        pm_flx = np.hstack((pm_flx, pm_flx_i))
        pm_err = np.hstack((pm_err, pm_err_i))
        tile_id = np.concatenate((tile_id, tile_id_i))
        pmra_sn = np.concatenate((pmra_sn, pmra_i))
        pmdec_sn = np.concatenate((pmdec_sn, pmdec_i))
        parallax_sn = np.concatenate((parallax_sn, parallax_i))
        starprob = np.concatenate((starprob, starprob_i))
        spCl = np.concatenate((spCl, spCl_i))
        zsp = np.concatenate((zsp, zsp_i))

    pm_err = (
        - flux_to_mag(pm_flx, w_central.reshape(-1, 1))
        + flux_to_mag(pm_flx - pm_err, w_central.reshape(-1, 1))
    )
    pm_flx = flux_to_mag(pm_flx, w_central.reshape(-1, 1))

    return pm_flx, pm_err, tile_id, pmra_sn, pmdec_sn, parallax_sn, starprob, N_minijpas,\
        spCl, zsp

flx, flx_err, tile_id, pmra_sn, pmdec_sn, parallax_sn, starprob, N_minijpas,\
    spCl, zsp = load_stuff()

In [ ]:
w_central = central_wavelength().reshape(-1, 1)
tcurves = load_tcurves(load_filter_tags())

In [ ]:
# Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_19nov_model11/Magnitudes_model_11/Qso_jpas_mock_mag_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12 - 1:-4] += 1
my_filter_order[28 - 2:-4] += 1
my_filter_order[43 - 3:-4] += 1

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
qso_zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Randomly sample sources corresponding to 200 deg2
idx = np.arange(100_000)

In [ ]:
# # Load my QSO catalog

# filename = '/home/alberto/almacen/Source_cats/QSO_100000_v4/data.csv'

# data = pd.read_csv(filename)
# qso_flx = data.to_numpy()[:, 1 : 60 + 1].T
# qso_err = np.abs(data.to_numpy()[:, 60 + 1 : 120 + 1].T)

# qso_err = -flux_to_mag(qso_flx + qso_err, w_central) + flux_to_mag(qso_flx, w_central)
# qso_flx = flux_to_mag(qso_flx, w_central)
# nanmag = np.isnan(qso_flx)
# qso_flx[nanmag] = 99.
# qso_err[nanmag] = 99.

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-4_v4/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = np.abs(data.to_numpy()[:, 60 + 1 : 120 + 1].T)

sf_err = -flux_to_mag(sf_flx + sf_err, w_central) + flux_to_mag(sf_flx, w_central)
sf_flx = flux_to_mag(sf_flx, w_central)
nanmag = np.isnan(sf_flx)
sf_flx[nanmag] = 99.
sf_err[nanmag] = 99.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

nb_c = 11
mask = (flx[nb_c] < 25)
qso_mask = (qso_flx[nb_c] < 25)
sf_mask = (sf_flx[nb_c] < 25)
ax.scatter(flx[nb_c, mask], flx_err[nb_c, mask], label='miniJPAS', zorder=99, alpha=0.2)
ax.scatter(qso_flx[nb_c, qso_mask], qso_err[nb_c, qso_mask], label='QSO mock', alpha=0.2)
# ax.scatter(sf_flx[nb_c, sf_mask], sf_err[nb_c, sf_mask], label='SF mock')

# x_Arr = np.linspace(1e-20, 1e-16, 100)

# ax.axvline(detec_lim[nb_c], ls='--', color='k')

ax.set_xlim((17, 24))
# ax.set_ylim((0, 1))
ax.set_ylim((1e-2, 1))

ax.legend(fontsize=15)

ax.set_xlabel('r (magAB)', fontsize=15)
ax.set_ylabel('$\sigma$(r) (magAB)', fontsize=15)
ax.set_title(tcurves['tag'][nb_c], fontsize=20)

ax.set_yscale('log')
# ax.set_xscale('log')

plt.show()